In [1]:
import numpy as np
import random

PERIOD_LENGTH = 6 # numero de horas de cada periodo
N_PERIODS_YEAR = int((24/PERIOD_LENGTH) * 365) # numero de periodos per year

# conjuntos
sources = ['solar', 'wind', 'hydroelectric', 'nuclear', 'gas', 'coal', 'oil'] # conjunto _i
periods = [i for i in range(N_PERIODS_YEAR)] # conjunto _t

# parametros constantes
budget = 1000000000 # presupuesto en dolares

years_return = 250 # cantidad de anos en los que se espera recuperar la inversion inicial

# TO-DOO
# demanda por periodo en kWh D_t
demand = {period:demand for period, demand in 
                  zip(periods, np.array([random.randint(5000, 15000) for _ in range(len(periods))]))}

battery_cost = 103240 # costo por bateria en dolares AC
battery_capacity = 232 # capacidad de almacenamiento en kWh por bateria AQ
battery_efficiency = 0.87 # eficiencia de energia de las baterias AE
pollution_per_battery = 46400 # kgs de CO2 emitidos por bateria producida AZ

kwh_price_mean = 0.1 # valor por kWh en el mercado promedio en dolares
kwh_price_chile = 0.16 # valor por kWh en chile en dolares


# parametro CI_i costo inicial por kW de capacidad en dolares
overnight_cost = {
    'solar': [1331, 1313, 1100],
    'wind': [1100, 1500, 1319],
    'hydroelectric': [5316, 1750, 7500, 3500],
    'nuclear': [6041, 6317, 6900, 12000],
    'gas': [950, 713, 1084, 1300],
    'coal': [3000, 6250, 5876, 3676],
    'oil': [1170, 1175, 713],
    }

# modelaremos ocupando la media en casos donde hayan mas de un dato
overnight_cost_mean = {
    key:np.mean(value) for key, value in  zip(overnight_cost.keys(), 
                                              overnight_cost.values())
}
    
# parametro C_i costo de produccion por kWh en dolares
production_cost = {
    'solar': [0.0017, 0.00136],
    'wind': [0.003, 0.0032],
    'hydroelectric': [0.005],
    'nuclear': [0.0210, 0.0138, 0.016],
    'gas': [0.021],
    'coal': [0.0098, 0.014, 0.0173],
    'oil': [0.054, 0.0701, 0.0563],
}

# al igual que el costo inicial, ocuparemos la media
production_cost_mean = {
    key:np.mean(value) for key, value in zip(production_cost.keys(),
                                            production_cost.values())
}

# kg de CO2 emitidos por producir un kWh Z_i
kg_CO2_per_kwh = {
    'solar': 0.085,
    'wind': 0.026,
    'hydroelectric': 0.026,
    'nuclear': 0.029,
    'gas': 0.499,
    'coal': 0.888,
    'oil': 0.733,
    }

# to-do: cantidad maxima disponible para instalaciones QM_i
prod_units_limit = {source:limit for source, limit in zip(sources, np.array([random.randint(500, 1500) for _ in
                                                                            range(len(sources))]))}

In [2]:
from gurobipy import GRB, Model, quicksum

model = Model('Energy production planning')

prod_units = model.addVars(sources, vtype=GRB.INTEGER, name='prod_units')
storage = model.addVar(vtype=GRB.INTEGER, name='storage')
output = model.addVars(periods, sources, vtype=GRB.CONTINUOUS, name='output')
stored = model.addVars(periods, vtype=GRB.CONTINUOUS, name='stored')

model.update()

Using license file /home/rieg/gurobi.lic
Academic license - for non-commercial use only


In [3]:
initial_investment = quicksum(prod_units[source] * overnight_cost_mean[source] for source in sources)

# restriccion 1: inversion inicial menor o igual a presupuesto
model.addConstr(initial_investment <= budget)

# restriccion 2: recuperacion de inversion (el presupuesto dividido en la cantidad de anos nos indica
# cuanto debemos recuperar cada ano)
revenue = quicksum(output[period, source] * (kwh_price_mean - production_cost_mean[source]) \
                     for source in sources for period in periods)
# revenue es lo que se vende multiplicado por el precio menos el costo variable
model.addConstr(revenue - (storage * battery_cost / years_return) -\
                (initial_investment / years_return) >= (budget/years_return))

# restriccion 3: satisfaccion de demanda y flujo
model.addConstrs(stored[period] == quicksum(output[period, source] for source in sources) - demand[period] + \
                                            (stored[period - 1] * battery_efficiency) \
                                                for idx, period in enumerate(periods) if idx != 0)

# restriccion 4: produccion de energia limitada
model.addConstrs(output[period, source] <= prod_units[source] * PERIOD_LENGTH for source in sources
                                                                                for period in periods)
# restriccion 5: cantidad de instalaciones limitadas 
model.addConstrs(prod_units[source] <= prod_units_limit[source] for source in sources)

# restriccion 6: cantidad maxima de almacenamiento
model.addConstrs(stored[period] <= storage * battery_capacity for period in periods)

# Naturaleza de las variables
# restriccion 7:
model.addConstrs(output[period, source] >= 0 for source in sources for period in periods)
# restriccion 8:
model.addConstrs(prod_units[source] >= 0 for source in sources)
# restriccion 9:
model.addConstr(storage >= 0)
# restriccion 10 y 11:
model.addConstrs(stored[period] >= 0 for idx, period in enumerate(periods) if idx != 0)
model.addConstrs(stored[period] == 0 for idx, period in enumerate(periods) if idx == 0)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>}

In [4]:
obj = quicksum(output[period, source] * kg_CO2_per_kwh[source]\
               for source in sources for period in periods) + storage * pollution_per_battery

In [5]:
model.setObjective(obj, GRB.MINIMIZE)

model.optimize()

model.printAttr('X')
print('\n---------------------\n')
for constr in model.getConstrs():
    print(constr, constr.getAttr('slack'))
    
model.write('model.sol')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 24836 rows, 11688 columns and 58421 nonzeros
Model fingerprint: 0x59263cea
Variable types: 11680 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 8e+03]
  Objective range  [3e-02, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+09]
Presolve removed 11697 rows and 1 columns
Presolve time: 0.02s
Presolved: 13139 rows, 11687 columns, 46716 nonzeros
Variable types: 11679 continuous, 8 integer (0 binary)

Root relaxation: objective 4.552181e+07, 4282 iterations, 0.56 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.5522e+07    0    2          - 4.5522e+07      -     -    0s
H    0     0                    5.814688e+07 4.5522e+07  21.7%     -    0s
H    0     0                    4.552226e+07 4.5522e+07  0.00%     -    0s

Explored 1

output[36,solar]         8424 
output[36,wind]         5244 
output[36,hydroelectric]         6480 
output[36,nuclear]         3282 
output[36,gas]         7956 
output[37,solar]         8424 
output[37,wind]         5244 
output[37,hydroelectric]         6480 
output[37,nuclear]         3282 
output[37,gas]       6971.2 
output[38,solar]         8424 
output[38,wind]         5244 
output[38,hydroelectric]         6480 
output[38,nuclear]         3282 
output[38,gas]         7956 
output[38,coal]        696.2 
output[39,solar]         8424 
output[39,wind]         5244 
output[39,hydroelectric]         6480 
output[39,nuclear]         3282 
output[39,gas]         7956 
output[39,coal]       2962.2 
output[40,solar]         8424 
output[40,wind]         5244 
output[40,hydroelectric]         6480 
output[40,nuclear]         3282 
output[40,gas]         7956 
output[40,coal]         4632 
output[41,solar]         8424 
output[41,wind]         5244 
output[41,hydroelectric]         6480 


output[82,nuclear]         3282 
output[82,gas]       6288.2 
output[83,solar]         8424 
output[83,wind]         5244 
output[83,hydroelectric]         6480 
output[83,nuclear]         3282 
output[83,gas]         7956 
output[83,coal]         4632 
output[84,solar]         8424 
output[84,wind]         5244 
output[84,hydroelectric]         6480 
output[84,nuclear]         3282 
output[84,gas]         7956 
output[84,coal]         4632 
output[85,solar]         8424 
output[85,wind]         5244 
output[85,hydroelectric]         6480 
output[85,nuclear]         3282 
output[85,gas]         7956 
output[85,coal]         4632 
output[86,solar]         8424 
output[86,wind]         5244 
output[86,hydroelectric]         6480 
output[86,nuclear]         3282 
output[86,gas]         7956 
output[87,solar]         8424 
output[87,wind]         5244 
output[87,hydroelectric]         6480 
output[87,nuclear]         3282 
output[87,gas]      7523.85 
output[88,solar]         8424 
output[

output[126,solar]         8424 
output[126,wind]         5244 
output[126,hydroelectric]         6480 
output[126,nuclear]         3282 
output[126,gas]         7956 
output[126,coal]         4632 
output[127,solar]         8424 
output[127,wind]         5244 
output[127,hydroelectric]         6480 
output[127,nuclear]         3282 
output[127,gas]         7956 
output[127,coal]         4632 
output[128,solar]         8424 
output[128,wind]         5244 
output[128,hydroelectric]         6480 
output[128,nuclear]         3282 
output[128,gas]         7956 
output[128,coal]         4632 
output[129,solar]         8424 
output[129,wind]         5244 
output[129,hydroelectric]         6480 
output[129,nuclear]         3282 
output[129,gas]         7956 
output[129,coal]         4632 
output[130,solar]         8424 
output[130,wind]         5244 
output[130,hydroelectric]         6480 
output[130,nuclear]         3282 
output[130,gas]         7956 
output[130,coal]      3947.21 
output[131

output[169,gas]         7956 
output[169,coal]         4632 
output[170,solar]         8424 
output[170,wind]         5244 
output[170,hydroelectric]         6480 
output[170,nuclear]         3282 
output[170,gas]         7956 
output[171,solar]         8424 
output[171,wind]         5244 
output[171,hydroelectric]         6480 
output[171,nuclear]         3282 
output[171,gas]      6617.09 
output[172,solar]         8424 
output[172,wind]         5244 
output[172,hydroelectric]         6480 
output[172,nuclear]         3282 
output[172,gas]         7956 
output[172,coal]      799.476 
output[173,solar]         8424 
output[173,wind]         5244 
output[173,hydroelectric]         6480 
output[173,nuclear]         3282 
output[173,gas]         7956 
output[174,solar]         8424 
output[174,wind]         5244 
output[174,hydroelectric]         6480 
output[174,nuclear]         3282 
output[174,gas]       6499.2 
output[175,solar]         8424 
output[175,wind]         5244 
output[175

...

output[1447,nuclear]         3282 
output[1447,gas]         7956 
output[1447,coal]      2688.33 
output[1448,solar]         8424 
output[1448,wind]         5244 
output[1448,hydroelectric]         6480 
output[1448,nuclear]         3282 
output[1448,gas]         7956 
output[1449,solar]         8424 
output[1449,wind]         5244 
output[1449,hydroelectric]         6480 
output[1449,nuclear]         3282 
output[1449,gas]      7034.63 
output[1450,solar]         8424 
output[1450,wind]         5244 
output[1450,hydroelectric]         6480 
output[1450,nuclear]         3282 
output[1450,gas]       4809.2 
output[1451,solar]         8424 
output[1451,wind]         5244 
output[1451,hydroelectric]         6480 
output[1451,nuclear]         3282 
output[1451,gas]       5067.2 
output[1452,solar]         8424 
output[1452,wind]         5244 
output[1452,hydroelectric]         6480 
output[1452,nuclear]         3282 
output[1452,gas]       5175.2 
output[1453,solar]         8424 
output[14

 stored[220]       178640 
 stored[221]       178099 
 stored[222]       178640 
 stored[223]       178640 
 stored[224]       178443 
 stored[225]       178640 
 stored[226]       178640 
 stored[227]       178640 
 stored[228]       178057 
 stored[229]       178640 
 stored[230]       178640 
 stored[231]       178640 
 stored[232]       178640 
 stored[233]       178640 
 stored[234]       178640 
 stored[235]       177049 
 stored[236]       178640 
 stored[237]       178640 
 stored[238]       178640 
 stored[239]       178568 
 stored[240]       178640 
 stored[241]       178640 
 stored[242]       178640 
 stored[243]       178640 
 stored[244]       178640 
 stored[245]       178640 
 stored[246]       178640 
 stored[247]       178640 
 stored[248]       178030 
 stored[249]       178640 
 stored[250]       178092 
 stored[251]       176134 
 stored[252]       174673 
 stored[253]       174885 
 stored[254]       176684 
 stored[255]       178640 
 stored[256]       176933 
 

 stored[524]       177607 
 stored[525]       178640 
 stored[526]       178444 
 stored[527]       177544 
 stored[528]       177816 
 stored[529]       178640 
 stored[530]       178640 
 stored[531]       178222 
 stored[532]       178640 
 stored[533]       178640 
 stored[534]       177141 
 stored[535]       175577 
 stored[536]       178640 
 stored[537]       178640 
 stored[538]       176489 
 stored[539]       178640 
 stored[540]       176105 
 stored[541]       178640 
 stored[542]       178640 
 stored[543]       178640 
 stored[544]       178640 
 stored[545]       178334 
 stored[546]       178640 
 stored[547]       178640 
 stored[548]       177408 
 stored[549]       178640 
 stored[550]       178640 
 stored[551]       178640 
 stored[552]       175126 
 stored[553]       178640 
 stored[554]       178640 
 stored[555]       176646 
 stored[556]       178640 
 stored[557]       178512 
 stored[558]       178640 
 stored[559]       178640 
 stored[560]       177892 
 

 stored[828]       178640 
 stored[829]       178640 
 stored[830]       177761 
 stored[831]       178640 
 stored[832]       178138 
 stored[833]       177762 
 stored[834]       178640 
 stored[835]       176478 
 stored[836]       178640 
 stored[837]       178640 
 stored[838]       178640 
 stored[839]       177730 
 stored[840]       178640 
 stored[841]       178640 
 stored[842]       178640 
 stored[843]       178640 
 stored[844]       178640 
 stored[845]       178640 
 stored[846]       178640 
 stored[847]       178640 
 stored[848]       178640 
 stored[849]       178640 
 stored[850]       175655 
 stored[851]       178640 
 stored[852]       178584 
 stored[853]       176500 
 stored[854]       178640 
 stored[855]       176921 
 stored[856]       177247 
 stored[857]       178640 
 stored[858]       178214 
 stored[859]       178640 
 stored[860]       178640 
 stored[861]       176844 
 stored[862]       178640 
 stored[863]       176618 
 stored[864]       178640 
 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
model.computeIIS()
model.write("model.ilp")


IIS runtime: 0.63 seconds


GurobiError: Cannot compute IIS on a feasible model

In [ ]:
if model.status == GRB.INFEASIBLE:
    model.feasRelaxS(1, False, False, True)
    model.optimize()